In [1]:
import time
import pandas as pd
import os
import pygame
import sys
import itertools
import pygame.mixer  # Importar el módulo de sonido

# Variables globales
CICLO = None
INTERVALO_PREGUNTA = None
TIEMPO_ORACION = None
TIEMPO_TARGET = None
TIEMPO_ENTRE_ESTIMULO = None
TIEMPO_ESTIMULO_PREGUNTA = None

def leer_configuracion(ruta):
    global CICLO, INTERVALO_PREGUNTA, TIEMPO_ORACION, TIEMPO_TARGET, TIEMPO_ENTRE_ESTIMULO, TIEMPO_ESTIMULO_PREGUNTA

    # Valores predeterminados
    CICLO = 1
    INTERVALO_PREGUNTA = 30
    TIEMPO_ORACION = 300
    TIEMPO_TARGET = 500
    TIEMPO_ENTRE_ESTIMULO = 300
    TIEMPO_ESTIMULO_PREGUNTA = 300

    with open(ruta, 'r') as archivo:
        for linea in archivo:
            try:
                nombre, valor = linea.strip().split(': ')
                if nombre == 'CICLO':
                    CICLO = int(valor)
                elif nombre == 'INTERVALO_PREGUNTA':
                    INTERVALO_PREGUNTA = int(valor)
                elif nombre == 'TIEMPO_ORACION':
                    TIEMPO_ORACION = int(valor)
                elif nombre == 'TIEMPO_TARGET':
                    TIEMPO_TARGET = int(valor) if valor else TIEMPO_TARGET
                elif nombre == 'TIEMPO_ENTRE_ESTIMULO':
                    TIEMPO_ENTRE_ESTIMULO = int(valor)
                elif nombre == 'TIEMPO_ESTIMULO_PREGUNTA':
                    TIEMPO_ESTIMULO_PREGUNTA = int(valor)
            except ValueError:
                print(f"Error en la línea: {linea}. Usando valor predeterminado.")

# Función para actualizar el archivo de configuración
def actualizar_configuracion(ruta, nombre, valor):
    with open(ruta, 'r') as archivo:
        lineas = archivo.readlines()

    with open(ruta, 'w') as archivo:
        for linea in lineas:
            if linea.startswith(nombre):
                archivo.write(f'{nombre}: {valor}\n')
            else:
                archivo.write(linea)

fields = {
            'Nombre':[],
            'Tiempo Inicio':[],
            'Tiempo Finalización':[],
            'Bloque':[],
            'Condición':[],
            'Genero':[],
            'Estimulo':[],
            'Respuesta correcta':[],
            'Respuesta dada':[],
            'Calificacion':[],
        }

# Clase que toma las medidas de la pantalla para ubicar las imágenes
class ScreenRect:
    origin = 0, 0
    scale = 1.0

    @classmethod
    def setGeometry(cls):
        screenRect = pygame.display.get_surface().get_rect()
        W, H = screenRect.size
        cls.scale = min(W, H)
        cls.origin = (W - cls.scale) / 2, (H - cls.scale) / 2

# Clase para mostrar palabras
class StimWord:
    pygame.font.init()
    font = pygame.font.Font(None, 70)
    color = pygame.Color('white')
    
    def __init__(self, word, max_width, max_height):
        self.word = word
        self.max_width = max_width
        self.max_height = max_height
        self.lines = self.wrap_text(word)
        self.images = [StimWord.font.render(line, True, StimWord.color) for line in self.lines]
        self.rects = [image.get_rect() for image in self.images]
        self.position_rects()

    def distributeGroups(self, file_path):
        
        # Cargar el archivo de Excel
        xls = pd.ExcelFile(file_path)

        # Crear una variable para cada hoja y extraer las columnas deseadas
        sheet1 = xls.parse(xls.sheet_names[0]).iloc[:, [8, 2, 3, 4, -2, -1]]  # Supone que AUDIOS_M y AUDIOS_F son las dos últimas columnas
        sheet2 = xls.parse(xls.sheet_names[1]).iloc[:, [8, 2, 3, 4, -2, -1]]
        sheet3 = xls.parse(xls.sheet_names[2]).iloc[:, [8, 2, 3, 4, -2, -1]]
        sheet4 = xls.parse(xls.sheet_names[3]).iloc[:, [8, 2, 3, 4, -2, -1]]

        # Crear listas para cada una de las cuatro condiciones
        grupo1 = []
        grupo2 = []
        grupo3 = []
        grupo4 = []

        # Número de elementos por condición
        num_elements = 160

        # Crear las listas combinadas
        for i in range(0, num_elements, 4):
            grupo1.extend([
                sheet1.iloc[i % len(sheet1)].tolist(),
                sheet2.iloc[(i + 1) % len(sheet2)].tolist(),
                sheet3.iloc[(i + 2) % len(sheet3)].tolist(),
                sheet4.iloc[(i + 3) % len(sheet4)].tolist()
            ])
            grupo2.extend([
                sheet1.iloc[(i + 1) % len(sheet1)].tolist(),
                sheet2.iloc[(i + 2) % len(sheet2)].tolist(),
                sheet3.iloc[(i + 3) % len(sheet3)].tolist(),
                sheet4.iloc[(i + 4) % len(sheet4)].tolist()
            ])
            grupo3.extend([
                sheet1.iloc[(i + 2) % len(sheet1)].tolist(),
                sheet2.iloc[(i + 3) % len(sheet2)].tolist(),
                sheet3.iloc[(i + 4) % len(sheet3)].tolist(),
                sheet4.iloc[(i + 5) % len(sheet4)].tolist()
            ])
            grupo4.extend([
                sheet1.iloc[(i + 3) % len(sheet1)].tolist(),
                sheet2.iloc[(i + 4) % len(sheet2)].tolist(),
                sheet3.iloc[(i + 5) % len(sheet3)].tolist(),
                sheet4.iloc[(i + 6) % len(sheet4)].tolist()
            ])

        return [grupo1, grupo2, grupo3, grupo4]


    def wrap_text(self, text):
        words = text.split(' ')
        lines = []
        current_line = []
        for word in words:
            current_line.append(word)
            line_width, _ = StimWord.font.size(' '.join(current_line))
            if line_width > self.max_width:
                current_line.pop()
                lines.append(' '.join(current_line))
                current_line = [word]
        lines.append(' '.join(current_line))
        return lines

    def position_rects(self):
        total_height = sum(rect.height for rect in self.rects)
        screen_rect = pygame.display.get_surface().get_rect()
        start_y = (screen_rect.height - total_height) // 2
        for rect in self.rects:
            rect.centerx = screen_rect.centerx
            rect.y = start_y
            start_y += rect.height

    def display(self):
        screen = pygame.display.get_surface()
        for image, rect in zip(self.images, self.rects):
            screen.blit(image, rect)
        pygame.display.update()


# Función para leer el archivo Excel
# def leer_excel(ruta):
#     df = pd.read_excel(ruta)
#     contexts = df['Contexto'].tolist()
#     phrase1 = df['Frase_1'].tolist()
#     phrase2 = df['Frase_2'].tolist()
#     audios = df['Audios'].tolist()  # Nueva columna para los audios
#     return contexts, phrase1, phrase2, audios

def leer_excel(ruta):
    distribuidor = StimWord('', 0, 0)  # Crear una instancia de StimWord para acceder a distributeGroups
    return map(list, zip(*itertools.chain(*distribuidor.distributeGroups(ruta))))

def read_questions(ruta):
    df = pd.read_excel(ruta)
    questions = df['PREGUNTAS'].tolist()
    return questions

# Función para agregar valores a cada lista
def agregar_valor(fields, key, valor):
    fields[key].append(valor)
    
    
    #agregar_valor(fields, 'Nombre', 'Nombre Ejemplo')
    #agregar_valor(fields, 'Tiempo Inicio', '2024-07-30 12:00:00')
    #agregar_valor(fields, 'Tiempo Finalización', '2024-07-30 12:05:00')
    #agregar_valor(fields, 'Bloque', 1)
    #agregar_valor(fields, 'Condición', 'Condición Ejemplo')
    #agregar_valor(fields, 'Genero', 'F')
    #agregar_valor(fields, 'Estimulo', 'Estimulo Ejemplo')
    #agregar_valor(fields, 'Respuesta correcta', 'Correcta')
    #agregar_valor(fields, 'Respuesta dada', 'Dada')
    #agregar_valor(fields, 'Calificacion', "Correcta")

# Clase principal ERT
class ERT:
    @staticmethod
    def run():

        global CICLO

        # Leer configuración al inicio del programa
        leer_configuracion('./CONFIG.txt')

        print(CICLO)
        print(INTERVALO_PREGUNTA)
        print(TIEMPO_ORACION)
        print(TIEMPO_TARGET)
        print(TIEMPO_ENTRE_ESTIMULO)
        print(TIEMPO_ESTIMULO_PREGUNTA)

        # Inicializar Pygame
        pygame.init()
        pygame.mixer.init()
        pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        pygame.mouse.set_visible(False)
        ScreenRect.setGeometry()

        # Leer los contexts y frases desde el archivo Excel
        type_stimulus, contexts, phrase1, phrase2, audiosM, audiosF = leer_excel(ruta='./4condiciones_160escenarios_EVA.xlsx')
        # contexts, phrase1, phrase2, audios = leer_excel('./Pruebas.xlsx')
        questions_all = read_questions("./Preguntas.xlsx")
        count_question = 1
        gender = 0        
        if (gender == 0):
            audios = audiosM
        else:
            audios = audiosF
        
        start = (int(CICLO) - 1) * 160
        end = start + 160

        contexts = contexts[start:end]
        phrase1 = phrase1[start:end]
        phrase2 = phrase2[start:end]
        audios = audios[start:end]

        print(audios)
        while True:
            for contexto, frase1, frase2, audio1 in zip(contexts, phrase1, phrase2, audios):
                
                
  
                # Mostrar el contexto
                pygame.display.get_surface().fill((0, 0, 0))
                contexto_display = StimWord(contexto, 800, 800)
                contexto_display.display()

                # Reproducir el audio y esperar a que se presione la tecla espacio
                pygame.mixer.music.load(audio1) 
                pygame.mixer.music.play()  # Reproducir el audio


                # Esperar a que se presione la tecla espacio
                esperando = True
                while esperando:
                    for evento in pygame.event.get():
                        if evento.type == pygame.KEYDOWN:
                            if evento.key == pygame.K_SPACE:
                                esperando = False
                            elif evento.key == pygame.K_ESCAPE:
                                # MARCA 1
                                pygame.quit()                                
                                sys.exit()
                

                

                # PANTALLA NEGRA PARA INICIAR FRASE FINAL
                pygame.display.get_surface().fill((0, 0, 0))
                frase_display = StimWord("", 800, 800)
                frase_display.display()
                pygame.time.wait(TIEMPO_ENTRE_ESTIMULO)

                sections = frase1.split()
                for frase in sections:
                    # FRASE FINAL POR PARTES
                    pygame.display.get_surface().fill((0, 0, 0))
                    frase_display = StimWord(frase, 800, 800)
                    frase_display.display()
                    pygame.time.wait(TIEMPO_ORACION)

                    pygame.display.get_surface().fill((0, 0, 0))
                    frase_display = StimWord("", 800, 800)
                    frase_display.display()
                    pygame.time.wait(TIEMPO_ENTRE_ESTIMULO)

                # TARGET
                pygame.display.get_surface().fill((0, 0, 0))
                frase_display = StimWord(frase2, 800, 800)
                frase_display.display()
                # Tiempo target
                pygame.time.wait(TIEMPO_TARGET)

                # PREGUNTA SI HAY QUE PREGUNTAR SEGUN LO DEFINIDO EN CONFIG.txt
                if (count_question % INTERVALO_PREGUNTA == 0):
                    
                    # PANTALLA NEGRA PARA INICIAR PREGUNTA
                    pygame.display.get_surface().fill((0, 0, 0))
                    frase_display = StimWord("", 800, 800)
                    frase_display.display()
                    pygame.time.wait(TIEMPO_ENTRE_ESTIMULO)
                    
                    # SE MUESTRA LA PREGUNTA
                    pygame.display.get_surface().fill((0, 0, 0))
                    frase_display = StimWord(questions_all[count_question - 1], 800, 800)
                    frase_display.display()

                    # Esperar a que se presione "z" o "m"
                    tecla_presionada = None
                    while tecla_presionada is None:
                        for evento in pygame.event.get():
                            if evento.type == pygame.KEYDOWN:
                                if evento.key == pygame.K_z or evento.key == pygame.K_m:
                                    tecla_presionada = pygame.key.name(evento.key)
                                elif evento.key == pygame.K_ESCAPE:
                                    pygame.quit()
                                    sys.exit()
                        
                    count_question += 1

                else:
                    count_question += 1

                # PANTALLA NEGRA ENTRE ESTIMULOS (CONTEXTOS)
                pygame.display.get_surface().fill((0, 0, 0))
                frase_display = StimWord("", 800, 800)
                frase_display.display()
                pygame.time.wait(TIEMPO_ENTRE_ESTIMULO)
                contexto_display.display()

        # Se actualiza el diccionario para guardar los datos en Excel
        df = pd.DataFrame(fields, columns=[
            'Nombre',
            'Tiempo Inicio',
            'Tiempo Finalización',
            'Bloque',
            'Condición',
            'Genero',
            'Estimulo',
            'Respuesta correcta',
            'Respuesta dada',
            'Calificacion'
        ])
        #carpeta = 'DATA'
        #nombre_archivo = Nombre+inicial+'.xlsx'
        #ruta_completa_p = os.path.join(carpeta, nombre_archivo)
        #df.to_excel(ruta_completa_p, sheet_name = "Conductual")

                # Espera
                pygame.time.wait(TIEMPO_ESTIMULO_PREGUNTA)
            CICLO += 1
            if (CICLO >= 5):
                CICLO = 1
            actualizar_configuracion('CONFIG.txt', 'CICLO', CICLO)
            


if __name__ == '__main__':
    ERT.run()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


UnboundLocalError: cannot access local variable 'CICLO' where it is not associated with a value

In [1]:
import pandas as pd


def distributeGroups(file_path):
        
    # Ruta del archivo de Excel
    # file_path = './4condiciones_160escenarios_EVA.xlsx'

    
    # Cargar el archivo de Excel
    xls = pd.ExcelFile(file_path)

    # Crear una variable para cada hoja y extraer las columnas deseadas
    sheet1 = xls.parse(xls.sheet_names[0]).iloc[:, [8, 2, 3, 4, -2, -1]]  # Supone que AUDIOS_M y AUDIOS_F son las dos últimas columnas
    sheet2 = xls.parse(xls.sheet_names[1]).iloc[:, [8, 2, 3, 4, -2, -1]]
    sheet3 = xls.parse(xls.sheet_names[2]).iloc[:, [8, 2, 3, 4, -2, -1]]
    sheet4 = xls.parse(xls.sheet_names[3]).iloc[:, [8, 2, 3, 4, -2, -1]]

    # Crear listas para cada una de las cuatro condiciones
    grupo1 = []
    grupo2 = []
    grupo3 = []
    grupo4 = []

    # Número de elementos por condición
    num_elements = 160

    # Crear las listas combinadas
    for i in range(0, num_elements, 4):
        grupo1.extend([
            sheet1.iloc[i % len(sheet1)].tolist(),
            sheet2.iloc[(i + 1) % len(sheet2)].tolist(),
            sheet3.iloc[(i + 2) % len(sheet3)].tolist(),
            sheet4.iloc[(i + 3) % len(sheet4)].tolist()
        ])
        grupo2.extend([
            sheet1.iloc[(i + 1) % len(sheet1)].tolist(),
            sheet2.iloc[(i + 2) % len(sheet2)].tolist(),
            sheet3.iloc[(i + 3) % len(sheet3)].tolist(),
            sheet4.iloc[(i + 4) % len(sheet4)].tolist()
        ])
        grupo3.extend([
            sheet1.iloc[(i + 2) % len(sheet1)].tolist(),
            sheet2.iloc[(i + 3) % len(sheet2)].tolist(),
            sheet3.iloc[(i + 4) % len(sheet3)].tolist(),
            sheet4.iloc[(i + 5) % len(sheet4)].tolist()
        ])
        grupo4.extend([
            sheet1.iloc[(i + 3) % len(sheet1)].tolist(),
            sheet2.iloc[(i + 4) % len(sheet2)].tolist(),
            sheet3.iloc[(i + 5) % len(sheet3)].tolist(),
            sheet4.iloc[(i + 6) % len(sheet4)].tolist()
        ])

    return [grupo1, grupo2, grupo3, grupo4]


In [1]:
import pandas as pd

# Ruta del archivo de Excel
file_path = './4condiciones_160escenarios_EVA.xlsx'

# Cargar el archivo de Excel
xls = pd.ExcelFile(file_path)

# Crear una variable para cada hoja y extraer las columnas deseadas
sheet1 = xls.parse(xls.sheet_names[0]).iloc[:, [8, 2, 3, 4, -2, -1]]  # Supone que AUDIOS_M y AUDIOS_F son las dos últimas columnas
sheet2 = xls.parse(xls.sheet_names[1]).iloc[:, [8, 2, 3, 4, -2, -1]]
sheet3 = xls.parse(xls.sheet_names[2]).iloc[:, [8, 2, 3, 4, -2, -1]]
sheet4 = xls.parse(xls.sheet_names[3]).iloc[:, [8, 2, 3, 4, -2, -1]]

# Crear listas para cada una de las cuatro condiciones
grupo1 = []
grupo2 = []
grupo3 = []
grupo4 = []

# Número de elementos por condición
num_elements = 160

# Crear las listas combinadas
for i in range(0, num_elements, 4):
    grupo1.extend([
        sheet1.iloc[i % len(sheet1)].tolist(),
        sheet2.iloc[(i + 1) % len(sheet2)].tolist(),
        sheet3.iloc[(i + 2) % len(sheet3)].tolist(),
        sheet4.iloc[(i + 3) % len(sheet4)].tolist()
    ])
    grupo2.extend([
        sheet1.iloc[(i + 1) % len(sheet1)].tolist(),
        sheet2.iloc[(i + 2) % len(sheet2)].tolist(),
        sheet3.iloc[(i + 3) % len(sheet3)].tolist(),
        sheet4.iloc[(i + 4) % len(sheet4)].tolist()
    ])
    grupo3.extend([
        sheet1.iloc[(i + 2) % len(sheet1)].tolist(),
        sheet2.iloc[(i + 3) % len(sheet2)].tolist(),
        sheet3.iloc[(i + 4) % len(sheet3)].tolist(),
        sheet4.iloc[(i + 5) % len(sheet4)].tolist()
    ])
    grupo4.extend([
        sheet1.iloc[(i + 3) % len(sheet1)].tolist(),
        sheet2.iloc[(i + 4) % len(sheet2)].tolist(),
        sheet3.iloc[(i + 5) % len(sheet3)].tolist(),
        sheet4.iloc[(i + 6) % len(sheet4)].tolist()
    ])

# Verificar el contenido de los grupos (opcional)
print("Grupo 1:", grupo1)
print("Grupo 2:", grupo2)
print("Grupo 3:", grupo3)
print("Grupo 4:", grupo4)


Grupo 1: [['Social  Alineada', 'Carlos es un chico dulce y sensible enamorado de una chica de su clase llamada Lucía. Es muy atento con ella y le recuerda lo guapa que va a clase todos los días. Un buen día, se arma de valor y la invita a salir.', 'Lucía le espera a la salida de clase y le da un ', 'beso.', './AUDIOS/ASM1.mp3', './AUDIOS/ASF1.mp3'], ['Social  Desalineada', 'Federico es el más listo de su clase. Siempre ayuda a sus compañeros con los deberes. Sobre todo, se preocupa por Roberto, un chico al que le van mal los estudios. Roberto está dejando de ir al colegio.', 'A Federico le llama el jefe de estudios y le ', 'expulsa.', './AUDIOS/ASM2.mp3', './AUDIOS/ASF2.mp3'], ['Antisocial Alineada', 'Carla es directora de una empresa con un futuro brillante. Carla comienza a saborear las mieles del éxito y se vuelve inflexible y dura con sus empleados. Los carga de trabajo y reduce su sueldo. En algunos casos, los despide sin previo aviso.', 'La empresa de Carla entra en ', 'bancarrot

In [2]:
groups = [grupo1, grupo2, grupo3, grupo4]


In [14]:
type_stimulus = []
context = []
phrase1 = []
phrase2 = []
for group in groups:
    for content in group:
   
        type_stimulus.append(content[0])
        context.append(content[1])
        phrase1.append(content[2])
        phrase2.append(content[3])

Social  Alineada
Carlos es un chico dulce y sensible enamorado de una chica de su clase llamada Lucía. Es muy atento con ella y le recuerda lo guapa que va a clase todos los días. Un buen día, se arma de valor y la invita a salir.
Lucía le espera a la salida de clase y le da un 
beso.
Social  Desalineada
Federico es el más listo de su clase. Siempre ayuda a sus compañeros con los deberes. Sobre todo, se preocupa por Roberto, un chico al que le van mal los estudios. Roberto está dejando de ir al colegio.
A Federico le llama el jefe de estudios y le 
expulsa.
Antisocial Alineada
Carla es directora de una empresa con un futuro brillante. Carla comienza a saborear las mieles del éxito y se vuelve inflexible y dura con sus empleados. Los carga de trabajo y reduce su sueldo. En algunos casos, los despide sin previo aviso.
La empresa de Carla entra en 
bancarrota.
Antisocial Desalineada
Javier es muy mandón. Siempre va de líder. Si alguien del grupo propone un plan, lo boicotea convenciendo a

In [3]:
import itertools

type_stimulus, context, phrase1, phrase2, audiosM, audiosF = map(list, zip(*itertools.chain(*groups)))


In [5]:
len(audiosF)

640